# MegaContext Research Console

Interactively prepare datasets, train GistNet with PyTorch Lightning, and capture experiment artifacts. Key docs: [GistNet](https://brandf.github.io/MegaContext/architecture/components/GistNet), [GistNet Training](https://brandf.github.io/MegaContext/architecture/components/GistNet%20Training), [Telemetry](https://brandf.github.io/MegaContext/ops/Telemetry), [Alternating Optimization](https://brandf.github.io/MegaContext/ops/Alternating%20Optimization).


## Quick Start




In [ ]:
import sys
from pathlib import Path

repo_url = 'https://github.com/brandf/MegaContext.git'
workspace = Path('/content/MegaContext')
if not workspace.exists():
    !git clone $repo_url $workspace
else:
    !git -C $workspace pull --ff-only
%cd /content/MegaContext

### Mount google drive (only on colab, uncomment below)

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

### Setup Dependencies (PIP)

In [ ]:
import importlib
import os
import sys
from pathlib import Path
from IPython import get_ipython

try:
    import ipywidgets as widgets  # type: ignore
except ImportError:  # pragma: no cover
    widgets = None

print('Python executable:', sys.executable)
shell = get_ipython()
if shell is None:
    raise RuntimeError('IPython shell not found; cannot run pip installs.')

QUIET_PIP = os.environ.get('MEGACONTEXT_PIP_QUIET', '1').strip().lower() not in {'0', 'false', 'no'}

def pip_install(args: str, *, quiet: bool | None = None) -> None:
    effective_quiet = QUIET_PIP if quiet is None else quiet
    extra_flags = ['--disable-pip-version-check']
    if effective_quiet and all(flag not in args for flag in ('-q', '--quiet')):
        extra_flags.insert(0, '--quiet')
    shell.run_line_magic('pip', f"{args} {' '.join(extra_flags)}")

def install_dependencies() -> None:
    print('Installing MegaContext dependencies...')
    for module_name in list(sys.modules):
        if module_name.startswith('megacontext') or module_name.startswith('lightning'):
            sys.modules.pop(module_name, None)
    pip_install('install --upgrade pip setuptools wheel')

    cuda_index = os.environ.get('PYTORCH_CUDA_INDEX', 'https://download.pytorch.org/whl/cu121')
    pip_install(f'install torch torchvision torchaudio --index-url {cuda_index}')
    pip_install('install lightning')
    pip_install('install -e .[dev]')
    pip_install('install rich>=13')
    pip_install('install tensorboard')
    # Auto-select widget version based on runtime (Colab vs JupyterLab).
    def _running_in_colab() -> bool:
        try:
            import google.colab  # type: ignore
        except ImportError:
            return False
        return True

    if _running_in_colab():
        pip_install('install ipywidgets==7.7.1')
    else:
        pip_install('install ipywidgets>=8.1,<9')
        pip_install('install jupyterlab-widgets>=3.0,<4')
    print('Widget target:', 'Colab 7.x' if _running_in_colab() else 'JupyterLab 8.x')

install_dependencies()
importlib.invalidate_caches()

src_path = workspace / 'src'
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))
try:
    import torch
    import megacontext  # noqa: F401
    import lightning  # noqa: F401
except ModuleNotFoundError as exc:
    print(f'Missing dependency ({exc}).')

print('Torch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())





## 0. Setup Console

Configure experiment settings, storage, logging, and reproducibility before running the pipeline.
print('Notebook bootstrap finished — widgets may require refresh if versions mismatch.')


In [ ]:
# @title
from pathlib import Path
import os
import random
import dataclasses
from copy import deepcopy

import numpy as np
import yaml
from IPython.display import Markdown, display

from megacontext.notebook import (
    MetricsTracker,
    build_logger,
    collect_environment_report,
    format_dataset_summary,
    format_training_config,
    format_training_summary,
    iter_config_sections,
    render_environment_report,
    save_experiment_summary,
)
from megacontext.gistnet import (
    BaseModelSettings,
    GistNetConfig,
    GistNetTrainingConfig,
)


try:
    import ipywidgets as widgets  # type: ignore
except ImportError:  # pragma: no cover
    widgets = None


def _resolve_path(value):
    if value in (None, ""):
        return None
    return Path(value).expanduser().resolve()


def launch_setup_console() -> None:
    global LOGGER_STATE
    if widgets is None:
        print('ipywidgets not available — falling back to defaults.')
        config_root = Path('configs')
        configs = sorted(config_root.glob('*.yaml'))
        if not configs:
            raise RuntimeError('No experiment configs found under `configs/`.')
        selected_config = configs[0]
        globals()['EXPERIMENT_CONFIG'] = selected_config
        globals()['experiment_cfg'] = yaml.safe_load(selected_config.read_text(encoding='utf-8'))
        env_report = collect_environment_report()
        print(render_environment_report(env_report))
        artifact_root = _resolve_path(os.environ.get('MEGACONTEXT_ARTIFACT_ROOT')) or (Path.cwd() / 'artifacts').resolve()
        artifact_root.mkdir(parents=True, exist_ok=True)
        globals()['ARTIFACT_ROOT'] = artifact_root
        os.environ['MEGACONTEXT_ARTIFACT_ROOT'] = str(artifact_root)
        LOGGER_STATE = {
            'selection': os.environ.get('MEGACONTEXT_LOGGER', 'none'),
            'project': os.environ.get('MEGACONTEXT_LOGGER_PROJECT', 'megacontext-poc'),
            'run_name': os.environ.get('MEGACONTEXT_LOGGER_RUN', ''),
        }
        seed = int(os.environ.get('MEGACONTEXT_SEED', 42))
        globals()['RUN_SEED'] = seed
        print(f'Selected config: {selected_config}')
        print(f'Artifact root: {artifact_root}')
        print(f'Seed: {seed}')
        return

    config_root = Path('configs')
    configs = sorted(config_root.glob('*.yaml'))
    if not configs:
        raise RuntimeError('No experiment configs found under `configs/`.')

    selected_config = globals().get('EXPERIMENT_CONFIG', configs[0])
    if isinstance(selected_config, str):
        selected_config = Path(selected_config)
    if selected_config not in configs:
        selected_config = configs[0]

    artifact_default = _resolve_path(os.environ.get('MEGACONTEXT_ARTIFACT_ROOT')) or (Path.cwd() / 'artifacts').resolve()
    artifact_root = _resolve_path(globals().get('ARTIFACT_ROOT')) or artifact_default
    artifact_root.mkdir(parents=True, exist_ok=True)
    os.environ['MEGACONTEXT_ARTIFACT_ROOT'] = str(artifact_root)
    globals()['ARTIFACT_ROOT'] = artifact_root

    data_root_initial = _resolve_path(os.environ.get('MEGACONTEXT_DATA_ROOT'))

    LOGGER_STATE = globals().get('LOGGER_STATE', {
        'selection': 'none',
        'project': 'megacontext-poc',
        'run_name': '',
    })
    LOGGER_STATE = {
        'selection': LOGGER_STATE.get('selection', 'none'),
        'project': LOGGER_STATE.get('project', 'megacontext-poc'),
        'run_name': LOGGER_STATE.get('run_name', ''),
    }

    training_preview = widgets.Output()
    training_controls_box = widgets.VBox()

    def _resolve_teacher_hidden_size() -> int | None:
        dataset_result = globals().get('DATASET_RESULT')
        if not dataset_result:
            return None
        splits = dataset_result.get('splits', {})
        split_name = globals().get('SPLIT_NAME')
        candidate = None
        if split_name and split_name in splits:
            candidate = splits[split_name]
        elif splits:
            first_key = next(iter(splits))
            candidate = splits[first_key]
        if not candidate:
            return None
        return candidate.get('teacher_hidden_size')

    def _build_training_configs() -> tuple[GistNetConfig | None, GistNetTrainingConfig | None, str | None]:
        experiment = globals().get('experiment_cfg')
        if not experiment:
            return None, None, 'No experiment configuration loaded.'
        gistnet_cfg = experiment.get('gistnet', {})
        if not gistnet_cfg:
            return None, None, 'Experiment config missing `gistnet` section.'
        model_dict = deepcopy(gistnet_cfg.get('model', {}))
        if model_dict.get('hidden_size') == 'auto':
            hidden_size = _resolve_teacher_hidden_size()
            if hidden_size:
                model_dict['hidden_size'] = hidden_size
        try:
            model_config = GistNetConfig(**model_dict)
        except Exception as exc:  # pragma: no cover - validation feedback
            return None, None, str(exc)
        training_dict = deepcopy(gistnet_cfg.get('training', {}))
        try:
            training_config = GistNetTrainingConfig.from_dict(training_dict)
        except Exception as exc:  # pragma: no cover - validation feedback
            return model_config, None, str(exc)
        if training_config.base_model is None:
            base_cfg = experiment.get('base_model')
            if base_cfg is not None:
                training_config = dataclasses.replace(
                    training_config, base_model=BaseModelSettings.from_dict(base_cfg)
                )
        return model_config, training_config, None

    def _render_training_preview(message: str | None = None) -> None:
        training_preview.clear_output()
        error = globals().get('TRAINING_CONFIG_ERROR')
        training_config = globals().get('TRAINING_CONFIG')
        with training_preview:
            if message:
                display(Markdown(message))
            elif error:
                display(Markdown(f"**Training config unavailable:** {error}"))
            elif training_config:
                display(Markdown(format_training_config(training_config)))
            else:
                display(Markdown('Training configuration will populate after dataset prep.'))

    def _rebuild_training_controls() -> None:
        if widgets is None:
            return
        error = globals().get('TRAINING_CONFIG_ERROR')
        training_config = globals().get('TRAINING_CONFIG')
        model_config = globals().get('MODEL_CONFIG')
        if error:
            training_controls_box.children = (
                widgets.HTML(
                    '<b>Training configuration unavailable.</b><br>'
                    'Resolve the configuration issues above or run dataset prep.'
                ),
            )
            return
        if training_config is None or model_config is None:
            training_controls_box.children = (
                widgets.HTML('Run dataset prep to populate training configuration.'),
            )
            return
        batch_slider = widgets.IntSlider(
            value=int(training_config.batch_size),
            min=1,
            max=max(1, training_config.batch_size * 2),
            step=1,
            description='Batch size:',
        )
        log_every_slider = widgets.IntSlider(
            value=max(1, training_config.log_every_n_steps),
            min=1,
            max=max(1, training_config.log_every_n_steps * 2),
            step=1,
            description='Log every:',
        )
        phase_controls = []
        phase_boxes = []
        for phase in training_config.phases:
            steps_slider = widgets.IntSlider(
                value=int(phase.max_steps),
                min=1,
                max=max(int(phase.max_steps), 100),
                step=1,
                description='Steps',
            )
            lr_slider = widgets.FloatLogSlider(
                value=float(phase.lr),
                base=10,
                min=-6,
                max=0,
                step=0.1,
                description='LR',
            )
            window_slider = widgets.IntSlider(
                value=int(phase.window_tokens),
                min=int(model_config.block_size),
                max=max(int(phase.window_tokens), int(model_config.block_size) * 64),
                step=int(model_config.block_size),
                description='Window',
            )
            phase_controls.append((phase, (steps_slider, lr_slider, window_slider)))
            phase_boxes.append(
                widgets.VBox([
                    widgets.HTML(f'<b>{phase.name}</b> ({phase.objective})'),
                    steps_slider,
                    lr_slider,
                    window_slider,
                ])
            )
        apply_button = widgets.Button(description='Apply overrides', button_style='success')
        overrides_output = widgets.Output()

        def _apply_overrides(_):
            current_config = globals().get('TRAINING_CONFIG')
            if current_config is None:
                return
            phases = []
            for base_phase, controls in phase_controls:
                steps_slider, lr_slider, window_slider = controls
                phases.append(
                    dataclasses.replace(
                        base_phase,
                        max_steps=int(steps_slider.value),
                        lr=float(lr_slider.value),
                        window_tokens=int(window_slider.value),
                    )
                )
            new_config = dataclasses.replace(
                current_config,
                batch_size=int(batch_slider.value),
                log_every_n_steps=int(log_every_slider.value),
                phases=tuple(phases),
            )
            globals()['TRAINING_CONFIG'] = new_config
            globals()['TRAINING_CONFIG_ERROR'] = None
            overrides_output.clear_output()
            with overrides_output:
                display(Markdown(format_training_config(new_config, heading='Updated Training Config')))
            _render_training_preview()
            update_summary('Applied training overrides.')

        apply_button.on_click(_apply_overrides)
        training_controls_box.children = tuple(
            [batch_slider, log_every_slider]
            + phase_boxes
            + [apply_button, overrides_output]
        )

    def _reset_training_settings(status: str | None = None) -> None:
        model_config, training_config, error = _build_training_configs()
        globals()['MODEL_CONFIG'] = model_config
        globals()['TRAINING_CONFIG'] = training_config
        globals()['TRAINING_CONFIG_ERROR'] = error
        _render_training_preview()
        _rebuild_training_controls()
        try:
            update_summary(status)
        except NameError:
            pass

    globals()['refresh_training_settings'] = _reset_training_settings


    env_report = collect_environment_report()
    env_output = widgets.Output()
    with env_output:
        display(Markdown(render_environment_report(env_report)))

    seed_initial = int(globals().get('RUN_SEED', os.environ.get('MEGACONTEXT_SEED', 42)))
    state = {
        'artifact_root': str(artifact_root),
        'data_root': str(data_root_initial) if data_root_initial else '',
        'resume_checkpoint': globals().get('RESUME_CHECKPOINT'),
        'seed': seed_initial,
    }

    config_dropdown = widgets.Dropdown(
        options=[(cfg.stem, str(cfg)) for cfg in configs],
        value=str(selected_config),
        description='Experiment:',
        layout=widgets.Layout(width='60%'),
    )
    config_sections_container = widgets.VBox()

    def _render_config_sections(cfg):
        sections = iter_config_sections(cfg)
        if not sections:
            config_sections_container.children = (
                widgets.HTML(value='<em>No experiment configuration found.</em>'),
            )
            return
        tab_children = []
        for title, payload, docs_link in sections:
            section_output = widgets.Output()
            with section_output:
                heading = f"### {title}"
                if docs_link:
                    heading = f"{heading} ([docs]({docs_link}))"
                yaml_dump = yaml.safe_dump(payload, sort_keys=False)
                display(Markdown(f"{heading}\n```yaml\n{yaml_dump}\n```"))
            tab_children.append(section_output)
        tabs = widgets.Tab(children=tab_children)
        for idx, (title, _payload, _docs_link) in enumerate(sections):
            tabs.set_title(idx, title)
        config_sections_container.children = (tabs,)

    def _load_config(path: Path) -> None:
        cfg = yaml.safe_load(path.read_text(encoding='utf-8'))
        globals()['EXPERIMENT_CONFIG'] = path
        globals()['experiment_cfg'] = cfg
        _render_config_sections(cfg)
        _reset_training_settings('Loaded experiment configuration.')


    def update_summary(status: str | None = None) -> None:
        summary_lines = [
            f"- **Config:** `{globals().get('EXPERIMENT_CONFIG')}`",
            f"- **Artifact root:** `{state['artifact_root']}`",
        ]
        if state['data_root']:
            summary_lines.append(f"- **Data root:** `{state['data_root']}`")
        else:
            summary_lines.append('- **Data root:** config-relative')
        summary_lines.extend([
            f"- **Logger:** {LOGGER_STATE.get('selection', 'none')}",
            f"- **Resume checkpoint:** `{state['resume_checkpoint']}`" if state['resume_checkpoint'] else '- **Resume checkpoint:** none',
            f"- **WANDB token:** {'set' if os.environ.get('WANDB_API_KEY') else 'not set'}",
            f"- **HF token:** {'set' if os.environ.get('HUGGINGFACE_HUB_TOKEN') else 'not set'}",
            f"- **Seed:** `{state['seed']}`",
        ])
        training_error = globals().get('TRAINING_CONFIG_ERROR')
        training_cfg = globals().get('TRAINING_CONFIG')
        if training_cfg:
            summary_lines.append(f'- **Batch size:** {training_cfg.batch_size}')
            summary_lines.append(f'- **Training phases:** {len(training_cfg.phases)}')
        elif training_error:
            summary_lines.append(f'- **Training config:** {training_error}')
        summary_output.clear_output()
        with summary_output:
            if status:
                display(Markdown(status))
            display(Markdown('\n'.join(summary_lines)))

    def _on_config_change(change):
        if change['name'] != 'value':
            return
        _load_config(Path(change['new']))

    config_dropdown.observe(_on_config_change, names='value')

    wandb_input = widgets.Text(
        value=os.environ.get('WANDB_API_KEY', ''),
        description='W&B token:',
        layout=widgets.Layout(width='70%'),
        placeholder='Optional',
    )
    hf_input = widgets.Text(
        value=os.environ.get('HUGGINGFACE_HUB_TOKEN', ''),
        description='HF token:',
        layout=widgets.Layout(width='70%'),
        placeholder='Optional',
    )

    def _sync_wandb(change):
        if change['name'] != 'value':
            return
        os.environ['WANDB_API_KEY'] = change['new'].strip()
        update_summary()

    def _sync_hf(change):
        if change['name'] != 'value':
            return
        os.environ['HUGGINGFACE_HUB_TOKEN'] = change['new'].strip()
        update_summary()

    wandb_input.observe(_sync_wandb, names='value')
    hf_input.observe(_sync_hf, names='value')

    artifact_input = widgets.Text(
        value=str(artifact_root),
        description='Artifact root:',
        layout=widgets.Layout(width='70%'),
    )
    artifact_apply = widgets.Button(description='Apply', button_style='info', icon='save')
    artifact_message = widgets.Output()

    data_input = widgets.Text(
        value=str(data_root_initial) if data_root_initial else '',
        description='Data root:',
        layout=widgets.Layout(width='70%'),
        placeholder='Optional — defaults to config-relative paths',
    )
    data_apply = widgets.Button(description='Apply', button_style='info', icon='save')
    data_message = widgets.Output()

    resume_status = widgets.HTML('Starting a new run.')
    resume_dropdown = widgets.Dropdown(
        description='Resume from:',
        layout=widgets.Layout(width='70%'),
    )

    summary_output = widgets.Output()
    _load_config(selected_config)

    def refresh_resume_options() -> None:
        runs_dir = Path(state['artifact_root']) / 'gistnet'
        runs_dir.mkdir(parents=True, exist_ok=True)
        globals()['RUNS_DIR'] = runs_dir
        checkpoints = sorted(
            runs_dir.glob('**/*.ckpt'),
            key=lambda p: p.stat().st_mtime,
            reverse=True,
        )
        options = [('Start fresh', '')]
        for path in checkpoints:
            try:
                label = str(path.relative_to(Path(state['artifact_root'])))
            except ValueError:
                label = str(path)
            options.append((label, str(path)))
        current = str(state['resume_checkpoint']) if state['resume_checkpoint'] else ''
        values = [value for _, value in options]
        if current not in values:
            current = ''
        resume_dropdown.options = options
        resume_dropdown.value = current
        if current:
            resume_status.value = f'Resuming from <code>{current}</code>'
        else:
            resume_status.value = 'Starting a new run.'

    def _apply_artifact_path(_=None) -> None:
        raw = artifact_input.value.strip()
        if not raw:
            artifact_input.value = state['artifact_root']
            update_summary('Artifact root cannot be empty; keeping current path.')
            return
        new_path = Path(raw).expanduser().resolve()
        if str(new_path) == state['artifact_root']:
            return
        new_path.mkdir(parents=True, exist_ok=True)
        state['artifact_root'] = str(new_path)
        globals()['ARTIFACT_ROOT'] = new_path
        os.environ['MEGACONTEXT_ARTIFACT_ROOT'] = str(new_path)
        with artifact_message:
            artifact_message.clear_output()
            display(Markdown(f'Artifacts stored under `{new_path}`.'))
        refresh_resume_options()
        update_summary('Artifact root updated.')

    artifact_apply.on_click(_apply_artifact_path)

    def _apply_data_path(_=None) -> None:
        raw = data_input.value.strip()
        if not raw:
            os.environ.pop('MEGACONTEXT_DATA_ROOT', None)
            state['data_root'] = ''
            with data_message:
                data_message.clear_output()
                display(Markdown('Using dataset paths relative to the experiment config.'))
            update_summary('Data root cleared.')
            return
        path = Path(raw).expanduser().resolve()
        if str(path) == state['data_root']:
            return
        path.mkdir(parents=True, exist_ok=True)
        os.environ['MEGACONTEXT_DATA_ROOT'] = str(path)
        state['data_root'] = str(path)
        with data_message:
            data_message.clear_output()
            display(Markdown(f'Dataset root override set to `{path}`.'))
        update_summary('Data root updated.')

    data_apply.on_click(_apply_data_path)

    def _on_resume_change(change):
        if change['name'] != 'value':
            return
        selected = change['new']
        state['resume_checkpoint'] = Path(selected) if selected else None
        globals()['RESUME_CHECKPOINT'] = state['resume_checkpoint']
        if selected:
            resume_status.value = f'Resuming from <code>{selected}</code>'
        else:
            resume_status.value = 'Starting a new run.'
        update_summary()

    resume_dropdown.observe(_on_resume_change, names='value')

    logger_dropdown = widgets.Dropdown(
        options=[('Disabled', 'none'), ('Weights & Biases', 'wandb')],
        value=LOGGER_STATE['selection'],
        description='Logger:',
        layout=widgets.Layout(width='50%'),
    )
    project_text = widgets.Text(
        value=LOGGER_STATE['project'],
        description='Project:',
        layout=widgets.Layout(width='50%'),
    )
    run_text = widgets.Text(
        value=LOGGER_STATE['run_name'],
        placeholder='auto',
        description='Run name:',
        layout=widgets.Layout(width='50%'),
    )
    logger_summary = widgets.HTML()

    def update_logger_state(_=None):
        LOGGER_STATE['selection'] = logger_dropdown.value
        LOGGER_STATE['project'] = project_text.value.strip()
        LOGGER_STATE['run_name'] = run_text.value.strip()
        logger_summary.value = (
            f"<b>Logger:</b> {LOGGER_STATE['selection']}<br>"
            f"<b>Project:</b> {LOGGER_STATE['project'] or '—'}<br>"
            f"<b>Run name:</b> {LOGGER_STATE['run_name'] or 'auto'}"
        )
        update_summary()

    for widget in (logger_dropdown, project_text, run_text):
        widget.observe(update_logger_state, names='value')
    update_logger_state()

    seed_input = widgets.IntText(
        value=seed_initial,
        description='Global seed:',
        layout=widgets.Layout(width='30%'),
    )
    apply_button = widgets.Button(description='Apply settings', button_style='success', icon='check')

    def apply_settings(_=None):
        try:
            seed = int(seed_input.value)
        except (TypeError, ValueError):
            seed_input.value = state['seed']
            update_summary('Seed must be an integer; reverting to the previous value.')
            return
        state['seed'] = seed
        globals()['RUN_SEED'] = seed
        os.environ['MEGACONTEXT_SEED'] = str(seed)
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        try:
            import torch
            torch.manual_seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed_all(seed)
                try:
                    torch.backends.cudnn.deterministic = True
                    torch.backends.cudnn.benchmark = False
                except AttributeError:
                    pass
        except ImportError:
            pass
        update_summary(f'Seeded runtime with `{seed}`.')

    apply_button.on_click(apply_settings)

    refresh_resume_options()
    update_summary()

    overview_box = widgets.VBox([env_output, summary_output])
    experiment_box = widgets.VBox([config_dropdown, config_sections_container])
    training_box = widgets.VBox([training_preview, training_controls_box])
    storage_box = widgets.VBox([
        widgets.HBox([artifact_input, artifact_apply]),
        artifact_message,
        widgets.HBox([data_input, data_apply]),
        data_message,
        resume_dropdown,
        resume_status,
    ])
    logging_box = widgets.VBox([logger_dropdown, project_text, run_text, logger_summary])
    credentials_box = widgets.VBox([wandb_input, hf_input])
    reproducibility_box = widgets.VBox([seed_input, apply_button])

    tabs = widgets.Tab(children=[overview_box, experiment_box, training_box, storage_box, logging_box, credentials_box, reproducibility_box])
    titles = ['Overview', 'Experiment', 'Training', 'Storage', 'Logging', 'Credentials', 'Reproducibility']
    for idx, title in enumerate(titles):
        tabs.set_title(idx, title)

    display(tabs)
    apply_settings()
    globals()['SETUP_CONSOLE'] = tabs
    print('Setup console ready — adjust settings via the tabs above.')
    return tabs


console_widget = launch_setup_console()
if console_widget is not None:
    globals()['SETUP_CONSOLE'] = console_widget








## 1. Dataset Preparation

Runs `tools.prepare_dataset.prepare_dataset_from_config` with tqdm progress bars. Skip this if the shard already exists.


In [ ]:
# @title
from pathlib import Path
import glob
import subprocess
import yaml
from tqdm.auto import tqdm


def ensure_dataset_assets(config_path: Path) -> None:
    raw_config = yaml.safe_load(config_path.read_text())
    dataset_cfg = raw_config.get('dataset', raw_config)
    base_dir = config_path.parent
    splits = dataset_cfg.get('splits', {})
    if isinstance(splits, dict):
        iterable = splits.items()
    else:
        iterable = enumerate(splits or [])
    missing_patterns = []
    for split_name, split in tqdm(iterable, desc='Checking dataset assets'):
        if not isinstance(split, dict):
            continue
        pattern = (base_dir / split['source']).expanduser()
        matches = glob.glob(str(pattern), recursive=True)
        if matches:
            continue
        if 'gutenberg' in str(pattern):
            print('Downloading Gutenberg subset (one-time download)...')
            script_path = Path('tools/download_gutenberg.sh')
            if not script_path.exists():
                raise FileNotFoundError(
                    'Expected tools/download_gutenberg.sh to exist. '
                    'Populate the Gutenberg corpus manually or restore the helper script.'
                )
            glob_str = str(pattern)
            stop_idx = len(glob_str)
            for token in ('*', '?', '['):
                idx = glob_str.find(token)
                if idx != -1:
                    stop_idx = min(stop_idx, idx)
            target_dir = Path(glob_str[:stop_idx]).resolve()
            if target_dir.suffix:
                target_dir = target_dir.parent
            target_dir.mkdir(parents=True, exist_ok=True)
            try:
                subprocess.check_call(['bash', str(script_path), str(target_dir)])
            except FileNotFoundError as err:
                raise RuntimeError('`bash` not found while running Gutenberg download script.') from err
            except subprocess.CalledProcessError as err:
                raise RuntimeError(
                    'Gutenberg download script failed. Inspect the output above or run "bash tools/download_gutenberg.sh" manually to diagnose.'
                ) from err
            matches = glob.glob(str(pattern), recursive=True)
            if matches:
                continue
        missing_patterns.append((split_name, pattern))
        print(f"No files matched pattern {pattern}")
    if missing_patterns:
        unresolved = '\n - '.join(f"{name}: {path}" for name, path in missing_patterns)
        raise FileNotFoundError(
            'Dataset assets missing. Patterns without matches:\n - ' + unresolved
        )


config_path_obj = EXPERIMENT_CONFIG if isinstance(EXPERIMENT_CONFIG, Path) else Path(EXPERIMENT_CONFIG)
ensure_dataset_assets(config_path_obj)









In [ ]:
# @title
import os
from pathlib import Path

import yaml
from IPython.display import Markdown, display

from megacontext.notebook import format_dataset_summary
from tools.prepare_dataset import load_dataset_config, prepare_dataset_from_config

try:
    from tqdm.auto import tqdm
except ImportError:  # pragma: no cover
    tqdm = None

CONFIG_MODEL = load_dataset_config(EXPERIMENT_CONFIG)


def _resolve_dataset_path(split_name: str) -> Path:
    split_cfg = CONFIG_MODEL.splits[split_name]
    output_path_cfg = Path(split_cfg.output_path)
    if output_path_cfg.is_absolute():
        return output_path_cfg
    base_dir = EXPERIMENT_CONFIG.parent
    default_output = (base_dir / output_path_cfg).resolve()
    data_root_override = os.environ.get('MEGACONTEXT_DATA_ROOT')
    if data_root_override:
        resolved_base = base_dir.resolve()
        repo_root = (
            resolved_base.parents[1]
            if len(resolved_base.parents) >= 2
            else resolved_base
        )
        try:
            relative = default_output.relative_to(repo_root)
        except ValueError:
            relative = default_output.name
        return (Path(data_root_override).expanduser().resolve() / relative).resolve()
    return default_output


expected_outputs = {name: _resolve_dataset_path(name) for name in CONFIG_MODEL.splits}
metadata_path = CONFIG_MODEL.metadata_path()
if metadata_path.suffix not in {'.yaml', '.yml'}:
    metadata_path = metadata_path.with_suffix('.yaml')
metadata_path = metadata_path.resolve()
force_rebuild = os.environ.get('MEGACONTEXT_FORCE_DATA_REBUILD', '').lower() in {'1', 'true', 'yes'}
missing_outputs = [name for name, path in expected_outputs.items() if not path.exists()]
metadata_missing = not metadata_path.exists()

if force_rebuild:
    print('Forcing dataset rebuild because MEGACONTEXT_FORCE_DATA_REBUILD is set.')
if missing_outputs:
    print('Dataset outputs missing for splits: ' + ', '.join(missing_outputs))
if metadata_missing and not (force_rebuild or missing_outputs):
    print(f'Metadata not found at {metadata_path}; regenerating dataset summaries.')

run_prepare = force_rebuild or bool(missing_outputs) or metadata_missing

DATASET_RESULT = None
if run_prepare:
    progress = None
    if tqdm is not None:
        progress = tqdm(total=1, desc='Preparing dataset', leave=False, bar_format='{l_bar}{bar}| {elapsed}')
    else:
        print('Preparing dataset (tqdm unavailable)...')
    try:
        DATASET_RESULT = prepare_dataset_from_config(EXPERIMENT_CONFIG)
    finally:
        if tqdm is not None and progress is not None:
            progress.update(1)
            progress.close()
else:
    print('Dataset shards already present; skipping prepare_dataset_from_config.')
    metadata = yaml.safe_load(metadata_path.read_text(encoding='utf-8')) or {}
    DATASET_RESULT = {
        'config_path': str(EXPERIMENT_CONFIG),
        'metadata_path': str(metadata_path),
        'teacher_dtype': metadata.get('teacher_dtype'),
        'splits': metadata.get('splits', {}) or {},
    }

if not DATASET_RESULT['splits']:
    print('Warning: dataset summaries are empty. Consider rebuilding the dataset.')

display(Markdown(format_dataset_summary(DATASET_RESULT['splits'])))
if callable(globals().get('refresh_training_settings')):
    refresh_training_settings('Updated training configuration after dataset prep.')


available_splits = list(CONFIG_MODEL.splits.keys())
default_split = 'train' if 'train' in available_splits else available_splits[0]
SPLIT_NAME = globals().get('SPLIT_NAME', default_split)
if SPLIT_NAME not in available_splits:
    SPLIT_NAME = default_split


def _set_dataset_path(split_name: str) -> Path:
    path = _resolve_dataset_path(split_name)
    globals()['DATASET_PATH'] = path
    DATASET_RESULT['dataset_path'] = str(path)
    return path


DATASET_PATH = _set_dataset_path(SPLIT_NAME)

widgets_ready = widgets is not None

if widgets_ready:
    split_dropdown = widgets.Dropdown(
        options=[(name, name) for name in available_splits],
        value=SPLIT_NAME,
        description='Split:',
        layout=widgets.Layout(width='40%'),
    )
    split_message = widgets.Output()

    def _sync_split(change):
        if change.get('name') != 'value':
            return
        selected = change['new']
        globals()['SPLIT_NAME'] = selected
        path = _set_dataset_path(selected)
        with split_message:
            split_message.clear_output()
            display(Markdown(f"Using split **{selected}** → `{path}`"))

    split_dropdown.observe(_sync_split, names='value')
    display(widgets.VBox([split_dropdown, split_message]))
    with split_message:
        split_message.clear_output()
        display(Markdown(f"Using split **{SPLIT_NAME}** → `{DATASET_PATH}`"))
else:
    print(f"Using split '{SPLIT_NAME}' at {DATASET_PATH}")





### Sample Example

Peek at the first prepared context to sanity-check tokens and horizons.


In [ ]:
# @title
import pyarrow.ipc as pa_ipc
from pathlib import Path
from IPython.display import Markdown, display

if not Path(DATASET_PATH).exists():
    missing_msg = (
        f"Dataset shard not found at {DATASET_PATH}. "
        "Run the prep cell first."
    )
    raise FileNotFoundError(missing_msg)

with pa_ipc.open_file(DATASET_PATH) as reader:
    if reader.num_record_batches == 0:
        print('Dataset is empty.')
    else:
        batch = reader.get_batch(0)
        table_dict = batch.to_pydict()
        context_col = 'context_input_ids'
        future_col = 'future_input_ids'
        missing = [col for col in (context_col, future_col) if col not in table_dict]
        if missing:
            display(Markdown(
                f"Dataset preview skipped; missing columns: {', '.join(missing)}"
            ))
        else:
            context_tokens = table_dict[context_col][0][:32]
            future_tokens = table_dict[future_col][0][:16]
            summary_html = (
                f"Context tokens (first 32): {context_tokens}<br><br>"
                f"Future tokens (first 16): {future_tokens}"
            )
            display(Markdown(summary_html))



## 2. GistNet Training

Two training phases run back-to-back:

- **Phase 1 – pooling MSE:** GistNet sees teacher hidden states for each 32-token block and learns to reproduce the block mean. This bootstraps the compression stack without querying the base LLM.
- **Phase 2 – ΔNLL distillation:** GistNet replaces those 32-token blocks with a single gist, feeds the mixed gist/token context through the frozen base model, and minimizes the increase in negative log-likelihood. Lower ΔNLL means the compressed gist is substitutable for the original span.

Check the live plot: the dashed 0.1 line marks the ΔNLL target for acceptable substitutability.


In [ ]:
# @title
from datetime import datetime
from pathlib import Path

from IPython.display import display
from lightning.pytorch.callbacks import ModelCheckpoint, RichProgressBar  # type: ignore
import torch

from megacontext.gistnet import build_gistnet_experiment
from megacontext.notebook import MetricsTracker

print('Configured torch float32 matmul precision to high for Tensor Cores.')
torch.set_float32_matmul_precision('high')
try:
    import ipywidgets as widgets  # type: ignore
except ImportError:
    widgets = None

resume_path = globals().get('RESUME_CHECKPOINT')
if isinstance(resume_path, str) and not resume_path:
    resume_path = None
if resume_path is not None and not isinstance(resume_path, Path):
    resume_path = Path(resume_path)
if resume_path is not None and not resume_path.exists():
    raise FileNotFoundError(f'Resume checkpoint not found: {resume_path}')

if resume_path:
    print(f'Resuming from checkpoint: {resume_path}')
else:
    print('Starting a new training run.')

live_metrics_output = None
if widgets is not None:
    try:
        live_metrics_output = widgets.Output()
        display(live_metrics_output)
    except Exception as exc:
        print('ipywidgets output unavailable; falling back to static plotting.', exc)
        live_metrics_output = None

plot_every = max(1, TRAINING_CONFIG.log_every_n_steps)
metrics_callback = MetricsTracker(
    metric_keys=(
        'train/loss',
        'train/delta_loss',
        'train/gist_loss',
        'train/baseline_loss',
    ),
    live_output=live_metrics_output,
    plot_every=plot_every,
    metric_labels={
        'train/loss': 'ΔNLL',
        'train/delta_loss': 'ΔNLL (explicit)',
        'train/gist_loss': 'Gist loss',
        'train/baseline_loss': 'Baseline loss',
    },
    reference_lines={
        'train/loss': 0.01,
        'train/delta_loss': 0.01,
    },
    y_label='Loss (nats)',
)
freeze_after_phase = len(TRAINING_CONFIG.phases) > 1

RUNS_DIR = globals().get('RUNS_DIR', (ARTIFACT_ROOT / 'gistnet').resolve())
RUNS_DIR.mkdir(parents=True, exist_ok=True)
run_name = (LOGGER_STATE.get('run_name') or '').strip()
if not run_name:
    run_name = f"{EXPERIMENT_CONFIG.stem}-{datetime.now():%Y%m%d-%H%M%S}"
run_name = run_name.replace(' ', '-')
RUN_NAME = run_name
RUN_DIR = (RUNS_DIR / run_name).resolve()
RUN_DIR.mkdir(parents=True, exist_ok=True)
CHECKPOINT_DIR = (RUN_DIR / 'checkpoints').resolve()
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
print(f'Artifacts for run `{RUN_NAME}` stored under {RUN_DIR}')

checkpoint_callback = ModelCheckpoint(
    dirpath=str(CHECKPOINT_DIR),
    filename='step-{step:06d}',
    monitor='train/loss',
    mode='min',
    save_last=True,
    save_top_k=3,
    every_n_train_steps=max(1, TRAINING_CONFIG.log_every_n_steps),
    auto_insert_metric_name=False,
)

callbacks = [metrics_callback, RichProgressBar(), checkpoint_callback]
LOGGER = None
try:
    LOGGER = build_logger(
        selection=LOGGER_STATE.get('selection', 'none'),
        project=LOGGER_STATE.get('project'),
        run_name=LOGGER_STATE.get('run_name'),
        config={'config_path': str(EXPERIMENT_CONFIG)},
    )
except RuntimeError as exc:
    print(exc)

trainer_kwargs = {
    'accelerator': 'auto',
    'devices': 1,
    'default_root_dir': str(RUN_DIR),
    'enable_model_summary': False,
}

TRAINER, MODULE, DATA_MODULE = build_gistnet_experiment(
    dataset_path=DATASET_PATH,
    model_config=MODEL_CONFIG,
    training=TRAINING_CONFIG,
    callbacks=callbacks,
    logger=LOGGER,
    trainer_kwargs=trainer_kwargs,
)

globals()['TRAINER'] = TRAINER
globals()['MODULE'] = MODULE
globals()['DATA_MODULE'] = DATA_MODULE
globals()['RUN_DIR'] = RUN_DIR
globals()['CHECKPOINT_DIR'] = CHECKPOINT_DIR
globals()['RUN_NAME'] = RUN_NAME

TRAINER.fit(MODULE, DATA_MODULE, ckpt_path=str(resume_path) if resume_path else None)

best_model_path = getattr(TRAINER.checkpoint_callback, 'best_model_path', None) or ''
if best_model_path:
    best_path = Path(best_model_path)
else:
    best_path = None
last_model_path = getattr(TRAINER.checkpoint_callback, 'last_model_path', None) or ''
if last_model_path:
    last_path = Path(last_model_path)
else:
    last_path = None
globals()['LATEST_CHECKPOINT'] = best_path
globals()['LAST_CHECKPOINT'] = last_path

if live_metrics_output is None:
    metrics_callback.plot()



## 3. Summarise & Save
Captures final metrics and writes a JSON summary under `artifacts/experiments/`.


In [ ]:
# @title
from pathlib import Path
from IPython.display import Markdown

metrics_callback.plot(figsize=(7, 4))

final_metrics = {k: float(v) for k, v in TRAINER.callback_metrics.items()}
if 'RUN_SEED' in globals():
    final_metrics.setdefault('seed', RUN_SEED)
display(Markdown(format_training_summary(final_metrics)))
seed_value = globals().get('RUN_SEED')
resume_path = globals().get('RESUME_CHECKPOINT')
run_dir = globals().get('RUN_DIR')
checkpoint_dir = globals().get('CHECKPOINT_DIR')
latest_checkpoint = globals().get('LATEST_CHECKPOINT')
last_checkpoint = globals().get('LAST_CHECKPOINT')
summary_root = (ARTIFACT_ROOT / 'experiments').resolve()
SUMMARY_PATH = save_experiment_summary(
    output_dir=summary_root,
    config_path=EXPERIMENT_CONFIG,
    dataset_summary=DATASET_RESULT['splits'],
    training_metrics=final_metrics,
    artifacts={
        'dataset_path': DATASET_RESULT.get('dataset_path'),
        'default_root_dir': str(run_dir) if run_dir else None,
        'checkpoint_dir': str(checkpoint_dir) if checkpoint_dir else None,
        'resume_from': str(resume_path) if resume_path else None,
        'latest_checkpoint': str(latest_checkpoint) if latest_checkpoint else None,
        'last_checkpoint': str(last_checkpoint) if last_checkpoint else None,
        'seed': seed_value,
    },
)
SUMMARY_PATH
